# Dependency Injection

The goal of this Jupyter Notebook is to learn about how to make a dependency injection in Python.

## What is Dependency Injection?

Dependency Injection is a software design pattern that allows us to decouple the creation of an object from its use. It is a way to implement the Inversion of Control (IoC) principle.

## How should a simple dependency injection look like in python?

```python
@Controller('cats')
class CatsController:
    catsService: CatsService
```

I should be able to inject the CatsService instance in the CatsController automatically without the need for a manual `__init__` method.

The injected class should depend on the type of the attribute.

In [88]:
def Injectable():
    def decorator(cls):
        def inner(*args, **kwargs):
            print("Injectable.inner ", args, kwargs)
            return NestProvider(cls)
        return inner
        # return lambda: NestProvider(cls)
    return decorator

def Controller(path=None):
    def decorator(cls):
        # return ControllerClass(cls)
        def inner():
            return ControllerClass(cls)
        return inner
    return decorator

class NestProvider:
    __name__ = 'NestProvider'
    # Note: this __providers__ should be automatically generated
    # by the module, but for this notebook I will leave it with
    # the providers manually
    __providers__ = {}

    @staticmethod
    def provide(name, instance):
        NestProvider.__providers__[name] = instance
    
    def __init__(self, cls):
        print(f"NestProvider {cls}")
        self.cls = cls()

    def __getattr__(self, name):
        if name in self.__providers__:
            return self.__providers__[name]
        return getattr(self.cls, name)
    
    def __call__(self, *args, **kwargs):
        return self.cls(self.cls, *args, **kwargs)

@Injectable()
class PersonClass:
    def printName(self):
        print('PersonClass.printName')

@Injectable()
class ControllerClass:
    __name__ = 'ControllerClass'

@Controller()
class PersonController:
    pass

pessoa = PersonClass()
pessoa.printName()


Injectable.inner  () {}
InjectableClass <class '__main__.PersonClass'>
PersonClass.printName


In [89]:
# POC test
@Injectable()
class CatsService:
    def getCatName(self):
        return "Kitty"


@Controller('cats')
class CatsController:
    catsService: CatsService

NestProvider.provide('catsService', CatsService())

print("Result: ", CatsController().catsService.getCatName())

Injectable.inner  () {}
InjectableClass <class '__main__.CatsService'>
Injectable.inner  (<class '__main__.CatsController'>,) {}
InjectableClass <class '__main__.ControllerClass'>
Result:  Kitty
